In [2]:
# 1. 라이브러리 불러오기
import pandas as pd
from konlpy.tag import Okt
import matplotlib.pyplot as plt
import seaborn as sns

# 한글 폰트 설정 (윈도우 기준)
plt.rcParams["font.family"] = "Malgun Gothic"
plt.rcParams["axes.unicode_minus"] = False

# 2. 데이터 불러오기
voc_df = pd.read_csv("./data/voc_data.csv", encoding="utf-8-sig")
inspection_df = pd.read_csv("./data/inspection_data.csv", encoding="utf-8-sig")
product_df = pd.read_csv("./data/product_data.csv", encoding="utf-8-sig")

# 3. 전처리 (형태소 분석 + 불용어 제거)
okt = Okt()
stopwords = ["이", "가", "은", "는", "에", "의", "도", "고", "와", "요", "있어요", "없어요", "해서"]
def preprocess_text(text):
    tokens = okt.morphs(text)
    return " ".join([t for t in tokens if t not in stopwords])
voc_df["전처리의견"] = voc_df["고객의견"].apply(preprocess_text)

# 4. 키워드 기반 분류
keyword_dict = {
    "정가품": ["정품", "가짜", "택", "인증"],
    "오염": ["오염", "얼룩", "먼지", "깨끗"],
    "포장불량": ["포장", "박스", "테이프", "엉망"],
    "배송지연": ["배송", "늦", "지연", "일주일"]
}
def classify_type(text):
    for label, keywords in keyword_dict.items():
        if any(k in text for k in keywords):
            return label
    return "기타"
voc_df["분류된유형"] = voc_df["전처리의견"].apply(classify_type)

# 5. VOC + 검수 merge (SKU, Date 기준)
merged_voc_inspection = pd.merge(voc_df, inspection_df, on=["SKU", "Date"], how="left")

# 6. product와 merge
merged_full = pd.merge(merged_voc_inspection, product_df, on="SKU", how="left")

# 7. 필요한 컬럼 정리
merged_full = merged_full[[
    "VOC_ID", "Date", "SKU", "브랜드", "카테고리", "매입수량",
    "고객의견", "전처리의견", "VOC유형", "분류된유형", "검수결과", "검수사유","감성점수"
]]

# 8. CSV 저장
merged_full.to_csv("./data/merged_voc_inspection_product.csv", index=False, encoding="utf-8-sig")


In [4]:
df = pd.read_csv("./data/merged_voc_inspection_product.csv", encoding="utf-8-sig")
print(df.columns)

Index(['VOC_ID', 'Date', 'SKU', '브랜드', '카테고리', '매입수량', '고객의견', '전처리의견',
       'VOC유형', '분류된유형', '검수결과', '검수사유', '감성점수'],
      dtype='object')
